In [1]:
import pandas as pd
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


In [2]:
df = pd.read_csv("E:\一带一路数据分析\一带一路四峰会.csv", encoding="ISO-8859-1")


In [3]:
df


,in_reply_to_username,retweeted_username,quoted_username,date,time,text,public_metrics.impression_count,public_metrics.reply_count,public_metrics.retweet_count,public_metrics.quote_count,...,author.location,author.public_metrics.followers_count,author.public_metrics.following_count,author.public_metrics.listed_count,author.public_metrics.tweet_count,author.verified,geo.country,geo.country_code,geo.full_name,geo.name
0,NaN,NaN,NaN,2017/5/7,0:42:30,Pakistan extends fully support to Chinese One ...,0,0,3,1,...,Islamabad,964108,14,1407,358280,True,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,2017/5/7,0:44:27,Economic approach of connectivity aims at elim...,0,0,3,0,...,Islamabad,964108,14,1407,358280,True,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,2017/5/7,2:35:20,Off to score on the Belt and Road@TokPa_Comms ...,0,0,1,0,...,Malaysia,129128,433,351,28326,True,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2017/5/7,3:00:11,.@arunjaitley expresses reservations on China¡...,0,0,0,0,...,"New Delhi, India",1049,200,37,9476,True,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2017/5/7,3:02:39,China's new Silk Road will be bumpy. https://t...,0,3,22,2,...,New York and the World,9210908,1477,70540,919225,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11486,NaN,NaN,NaN,2021/6/30,21:27:52,At the Asia and Pacific High-level Conference ...,0,1,1,0,...,NaN,42661,44,374,4223,True,NaN,NaN,NaN,NaN
11487,NaN,NaN,NaN,2021/6/30,21:31:48,At the Asia and Pacific High-level Conference ...,0,6,0,0,...,NaN,42661,44,374,4223,True,NaN,NaN,NaN,NaN
11488,ChineseEmbinUK,NaN,NaN,2021/6/30,21:31:49,Initiative for Belt and Road Partnership on CO...,0,5,0,0,...,NaN,42661,44,374,4223,True,NaN,NaN,NaN,NaN
11489,NaN,NikkeiAsia,NaN,2021/6/30,22:25:06,SRI LANKA &amp; CHINA | A BRIDGE TOO FAR?\n\nC...,0,0,3,0,...,|¾© Ç§´úÌïÇø,5499,57,123,8046,True,NaN,NaN,NaN,NaN


In [4]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopwords_list = stopwords.words("english")
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()


In [5]:
df["text"] = df["text"].str.lower()  # 统一为小写字母


In [6]:
# 定义一个函数，处理Twitter文本中的特殊符号：@xxx、&xxx、网址、多余的空格，最后只保留数字和字母
def clean_specialtext(tweet):
    tweet = re.sub(r"@([^\s:]+)", " ", tweet)  # 去除@提及内容
    tweet = re.sub(r"&([^\s:]+)", " ", tweet)  # 去除&及之后的内容
    tweet = re.sub(r"[a-zA-z]+://[^\s]*", " ", tweet)  # 去除网址
    tweet = re.sub(r"[^0-9A-Za-z]", " ", tweet)  # 仅保留数字及字母
    tweet = re.sub(r"(^\s*)|(\s*$)", "", tweet)  # 去除字符串前后空格
    # tweet = re.sub(r'[^0-9A-Za-z]', ' ', tweet)
    return tweet


In [7]:
df["text"] = df["text"].apply(lambda x: clean_specialtext(x))


In [8]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return None


In [9]:
# 定义去除停用词和词形还原函数，每一步都是有顺序的，不能乱
def clean_stopwords_Lemmatization(tweet):
    tweet = word_tokenize(tweet)
    # print(tweet)
    # print("*"*100)
    tweet = [word for word in tweet if word not in stopwords_list]  # 去除停用词
    tweet = pos_tag(tweet)  # 词性标注
    # 词性标注好之后，列表里变成了（词，词性）的元组形式，word[0]是词的位置，对word[0]进行词形还原，依据是上面的函数和word[1]
    tweet = [
        lemmatizer.lemmatize(word[0], pos=get_wordnet_pos(word[1]))
        if get_wordnet_pos(word[1])
        else word[0]
        for word in tweet
    ]
    tweet = [word for word in tweet if len(word) > 2]  # 丢弃长度小于等于2的字符串
    tweet = " ".join(tweet)  # 将tweet列表中的字符串重新拼接起来，用空格分隔
    # print(tweet)
    # print("#"*100)
    return tweet


In [10]:
df["text"] = df["text"].apply(lambda x: clean_stopwords_Lemmatization(x))


In [26]:
df["text"].str.replace("belt road", "bri")


0        pakistan extend fully support chinese one belt...
1        economic approach connectivity aim eliminate p...
2                                    score bri mitimatters
3        express reservation china one belt one road pr...
4                            china new silk road bumpy via
                               ...                        
11486    asia pacific high level conference bri coopera...
11487    asia pacific high level conference bri coopera...
11488    initiative bri partnership covid vaccine bri p...
11489    sri lanka china bridge far nchina facto contro...
11490    china facto control high profile sri lankan in...
Name: text, Length: 11491, dtype: object

In [30]:
df["text"] = (
    df["text"]
    .str.replace("belt road", "bri")
    .str.replace("one belt one road", "bri")
    .str.replace("new silk road", "bri")
)


In [ ]:
df_stage1 = 

In [1]:
df["date"].astype("datetime64[ns]")
df["2017-5-22" <= df["date"] >= "2017-5-7"]


NameError: name 'df' is not defined